In [ ]:
# @title Exp:4. Implementing of Denoising, sparse and contractive autoencoders.
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras.datasets.mnist import load_data
(X_train, y_train), (X_test, y_test) = load_data()
for i in range(4):
  plt.subplot(2, 2, i + 1)
plt.imshow(X_train[i], cmap=plt.cm.binary)
X_train, X_test = X_train / 255., X_test / 255.
X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))
factor = .3
X_train_noisy = X_train + factor * np.random.normal(loc=0.0, scale=1.,
size=X_train.shape)
X_test_noisy = X_test + factor * np.random.normal(loc=0.0, scale=1.,
size=X_test.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)
for i in range(4):
  plt.subplot(2, 2, i + 1)
plt.imshow(X_train_noisy[i].reshape(1, 28, 28)[0],
cmap=plt.cm.binary)
common_param = {
'padding': 'same',
'activation': 'relu'
}
model = Sequential([
# Encoder
Conv2D(16, (3, 3), **common_param, input_shape=(28, 28, 1)),
BatchNormalization(),
Conv2D(32, (3, 3), **common_param),
BatchNormalization(),
Conv2D(64, (3, 3), **common_param),
# Decoder
Conv2DTranspose(64, (3, 3), **common_param),
BatchNormalization(),
Conv2DTranspose(32, (3, 3), **common_param),
BatchNormalization(),
Conv2DTranspose(16, (3, 3), **common_param),
Conv2D(1, (3, 3), **common_param)
])
model.compile(optimizer='adam', loss='mean_squared_error',
metrics=['accuracy'])
model.summary()
history = model.fit(X_train_noisy, X_train, batch_size=256, epochs=5,
validation_data=(X_test_noisy, X_test))
plt.subplot(2,1,1)
plt.plot( history.history['loss'], label = 'loss')
plt.plot( history.history['val_loss'], label = 'val_loss')
plt.legend(loc = 'best')
plt.subplot(2,1,2)
plt.plot( history.history['accuracy'], label = 'accuracy')
plt.plot( history.history['val_accuracy'], label = 'val_accuracy')
plt.legend(loc = 'best')
plt.title('Model Evaluation Curve')
plt.show()
plt.figure(figsize=(10, 10))
for i in range(5):
  plt.subplot(5, 5, i + 1)
if i == 2:
  plt.title("Real Images", color='green')
plt.imshow(X_test[i].reshape(1, 28, 28)[0], cmap=plt.cm.binary)
plt.figure(figsize=(10, 10))
for i in range(5):
  plt.subplot(5, 5, i + 1)
if i == 2:
  plt.title("Noised Images", color='red')
plt.imshow(X_test_noisy[i].reshape(1, 28, 28)[0], cmap=plt.cm.binary)
plt.figure(figsize=(10, 10))
for i in range(5):
  plt.subplot(5, 5, i + 1)
if i == 2:
  plt.title("Denoised Images", color='blue')
plt.imshow(model.predict(X_test_noisy[i].reshape(1, 28, 28,
1)).reshape(1, 28, 28)[0], cmap=plt.cm.binary)